### Importando as Bibliotecas e Funções

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

def plot_profit_acu(dataframe, title_text):
    dataframe['Profit_acu'] = dataframe.Profit.cumsum()
    dataframe['Investimento'] = 1
    n_apostas = dataframe.shape[0]
    profit = round(dataframe.Profit_acu.tail(1).item(), 2)
    dataframe['Investimento_acu'] = dataframe.Investimento.cumsum()
    ROI = round(((dataframe.Profit_acu.tail(1) / dataframe.Investimento_acu.tail(1)) * 100).item(), 2)
    drawdown = dataframe['Profit_acu'] - dataframe['Profit_acu'].cummax()
    drawdown_maximo = round(drawdown.min(), 2)
    winrate_medio = round((dataframe['Profit'] > 0).mean() * 100, 2)
    desvio_padrao = round(dataframe['Profit'].std(), 2)

    ax = dataframe.Profit_acu.plot(title=title_text, xlabel='Entradas', ylabel='Stakes')
    ax.set_title(title_text)
    ax.set_xlabel('Entradas')
    ax.set_ylabel('Stakes')

    print("Metodo:", title_text)
    print("Profit:", profit, "stakes em", n_apostas, "jogos")
    print("ROI:", ROI, "%")
    print("Drawdown Maximo Acumulado:", drawdown_maximo)
    print("Winrate Medio:", winrate_medio, "%")
    print("Desvio Padrao:", desvio_padrao)
    print("")

    plt.show()

### Importando a Base de Dados

In [ ]:
df = pd.read_csv('https://github.com/futpythontrader/YouTube/raw/refs/heads/main/Bases_de_Dados/FootyStats/Base_de_Dados_FootyStats_(2022_2024).csv')
df = df[['League','Season','Date','Rodada','Home','Away',
         'Goals_H_FT','Goals_A_FT','TotalGoals_FT','Goals_H_Minutes','Goals_A_Minutes',
         'Odd_H_FT','Odd_D_FT','Odd_A_FT','Odd_Over25_FT','Odd_Under25_FT','Odd_BTTS_Yes','Odd_BTTS_No','Odd_DC_1X','Odd_DC_12','Odd_DC_X2']]
df = drop_reset_index(df)
display(df)

In [ ]:
columns_to_filter = [
    'Odd_Over25_FT', 'Odd_Under25_FT', 'Odd_BTTS_Yes',
    'Odd_BTTS_No', 'Odd_DC_1X', 'Odd_DC_12', 'Odd_DC_X2'
]
df = df[(df[columns_to_filter] != 0).all(axis=1)]
df = drop_reset_index(df)

display(df)

### Profits dos Mercados de MO, OU25 e BTTS

In [ ]:
df['Back_Home'] = np.where((df['Goals_H_FT'] > df['Goals_A_FT']), 1, 0)
df['Profit_Back_Home'] = np.where((df['Back_Home'] == 1), df['Odd_H_FT']-1, -1)

df['Back_Draw'] = np.where((df['Goals_H_FT'] == df['Goals_A_FT']), 1, 0)
df['Profit_Back_Draw'] = np.where((df['Back_Draw'] == 1), df['Odd_D_FT']-1, -1)

df['Back_Away'] = np.where((df['Goals_H_FT'] < df['Goals_A_FT']), 1, 0)
df['Profit_Back_Away'] = np.where((df['Back_Away'] == 1), df['Odd_A_FT']-1, -1)

display(df)

In [ ]:
df['Lay_Home'] = np.where((df['Goals_H_FT'] <= df['Goals_A_FT']), 1, 0)
df['Profit_Lay_Home'] = np.where((df['Lay_Home'] == 1), df['Odd_DC_X2']-1, -1)

df['Lay_Draw'] = np.where((df['Goals_H_FT'] != df['Goals_A_FT']), 1, 0)
df['Profit_Lay_Draw'] = np.where((df['Lay_Draw'] == 1), df['Odd_DC_12']-1, -1)

df['Lay_Away'] = np.where((df['Goals_H_FT'] >= df['Goals_A_FT']), 1, 0)
df['Profit_Lay_Away'] = np.where((df['Lay_Away'] == 1), df['Odd_DC_1X']-1, -1)

display(df)

In [ ]:
df['Over_25'] = np.where((df['TotalGoals_FT'] > 2), 1, 0)
df['Profit_Over_25'] = np.where((df['Over_25'] == 1), df['Odd_Over25_FT']-1, -1)

df['Under_25'] = np.where((df['TotalGoals_FT'] < 3), 1, 0)
df['Profit_Under_25'] = np.where((df['Under_25'] == 1), df['Odd_Under25_FT']-1, -1)

display(df)

In [ ]:
df['BTTS_Yes'] = np.where(((df['Goals_H_FT'] > 0) & (df['Goals_A_FT'] > 0)), 1, 0)
df['Profit_BTTS_Yes'] = np.where((df['BTTS_Yes'] == 1), df['Odd_BTTS_Yes']-1, -1)

df['BTTS_No'] = np.where(((df['Goals_H_FT'] == 0) | (df['Goals_A_FT'] == 0)), 1, 0)
df['Profit_BTTS_No'] = np.where((df['BTTS_No'] == 1), df['Odd_BTTS_No']-1, -1)

display(df)

In [ ]:
# modelo = 'Back_Home'

# df['Profit'] = df[f'Profit_{modelo}']
# plot_profit_acu(df, f'{modelo}')

### Probabilidades, CV MO, Variáveis Derivadas das Odds

##### Probabilidades

In [ ]:
df['p_H'] = 1 / df['Odd_H_FT']
df['p_D'] = 1 / df['Odd_D_FT']
df['p_A'] = 1 / df['Odd_A_FT']

##### Coeficiente de Variação das Odds do Match Odds

In [ ]:
desvio_padrao = df[['Odd_H_FT','Odd_D_FT','Odd_A_FT']].std(axis=1)
media = df[['Odd_H_FT','Odd_D_FT','Odd_A_FT']].mean(axis=1)
CV_Odds = desvio_padrao / media
df['CV_Odds'] = CV_Odds

##### Proporção entre as Odds

In [ ]:
df['H_D'] = df['Odd_H_FT'] / df['Odd_D_FT']
df['H_A'] = df['Odd_H_FT'] / df['Odd_A_FT']
df['D_H'] = df['Odd_D_FT'] / df['Odd_H_FT']
df['D_A'] = df['Odd_D_FT'] / df['Odd_A_FT']
df['A_H'] = df['Odd_A_FT'] / df['Odd_H_FT']
df['A_D'] = df['Odd_A_FT'] / df['Odd_D_FT']

##### Diferença Absoluta entre as Odds

In [ ]:
df['DifAbs_HomeAway'] = np.abs(df['Odd_H_FT'] - df['Odd_A_FT'])
df['DifAbs_HomeDraw'] = np.abs(df['Odd_H_FT'] - df['Odd_D_FT'])
df['DifAbs_DrawAway'] = np.abs(df['Odd_D_FT'] - df['Odd_A_FT'])

##### Angulo de Disparidade entre as Odds

In [ ]:
df['Angle_HomeAway'] = np.degrees(np.arctan((df['Odd_A_FT'] - df['Odd_H_FT']) / 2))
df['Angle_HomeDraw'] = np.degrees(np.arctan((df['Odd_D_FT'] - df['Odd_H_FT']) / 2))
df['Angle_DrawAway'] = np.degrees(np.arctan((df['Odd_A_FT'] - df['Odd_D_FT']) / 2))

##### Diferenças Percentuais entre as Odds

In [ ]:
df['DifPer_HomeAway'] = np.abs((df['Odd_H_FT'] - df['Odd_A_FT'])) / df['Odd_A_FT']
df['DifPer_HomeDraw'] = np.abs((df['Odd_H_FT'] - df['Odd_D_FT'])) / df['Odd_D_FT']
df['DifPer_DrawAway'] = np.abs((df['Odd_D_FT'] - df['Odd_A_FT'])) / df['Odd_A_FT']

In [ ]:
display(df)

### Variáveis de Desempenho

##### Média de Pontos (PPG)

In [ ]:
n_per = 5

In [ ]:
df['Ptos_H'] = np.where(df['Goals_H_FT'] >  df['Goals_A_FT'], 3,
               np.where(df['Goals_H_FT'] == df['Goals_A_FT'], 1, 0))


df['Ptos_A'] = np.where(df['Goals_H_FT'] >  df['Goals_A_FT'], 0,
               np.where(df['Goals_H_FT'] == df['Goals_A_FT'], 1, 3))

df['Media_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_Ptos_H'] = df.groupby('Home')['Media_Ptos_H'].shift(1)
df['Media_Ptos_A'] = df.groupby('Away')['Media_Ptos_A'].shift(1)

df['DesvPad_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_Ptos_H'] = df.groupby('Home')['DesvPad_Ptos_H'].shift(1)
df['DesvPad_Ptos_A'] = df.groupby('Away')['DesvPad_Ptos_A'].shift(1)

df['CV_Ptos_H'] = df['DesvPad_Ptos_H'] / df['Media_Ptos_H']
df['CV_Ptos_A'] = df['DesvPad_Ptos_A'] / df['Media_Ptos_A']

##### Média Gols Marcados

In [ ]:
df['Media_GM_H'] = df.groupby('Home')['Goals_H_FT'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GM_A'] = df.groupby('Away')['Goals_A_FT'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GM_H'] = df.groupby('Home')['Media_GM_H'].shift(1)
df['Media_GM_A'] = df.groupby('Away')['Media_GM_A'].shift(1)

df['DesvPad_GM_H'] = df.groupby('Home')['Goals_H_FT'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GM_A'] = df.groupby('Away')['Goals_A_FT'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GM_H'] = df.groupby('Home')['DesvPad_GM_H'].shift(1)
df['DesvPad_GM_A'] = df.groupby('Away')['DesvPad_GM_A'].shift(1)

df['CV_GM_H'] = df['DesvPad_GM_H'] / df['Media_GM_H']
df['CV_GM_A'] = df['DesvPad_GM_A'] / df['Media_GM_A']

##### Média de Gols Sofridos

In [ ]:
df['Media_GS_H'] = df.groupby('Home')['Goals_A_FT'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GS_A'] = df.groupby('Away')['Goals_H_FT'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GS_H'] = df.groupby('Home')['Media_GS_H'].shift(1)
df['Media_GS_A'] = df.groupby('Away')['Media_GS_A'].shift(1)

df['DesvPad_GS_H'] = df.groupby('Home')['Goals_A_FT'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GS_A'] = df.groupby('Away')['Goals_H_FT'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GS_H'] = df.groupby('Home')['DesvPad_GS_H'].shift(1)
df['DesvPad_GS_A'] = df.groupby('Away')['DesvPad_GS_A'].shift(1)

df['CV_GS_H'] = df['DesvPad_GS_H'] / df['Media_GS_H']
df['CV_GS_A'] = df['DesvPad_GS_A'] / df['Media_GS_A']

##### Média de Saldo de Gols

In [ ]:
df['SG_H'] = df['Goals_H_FT'] - df['Goals_A_FT']
df['SG_A'] = df['Goals_A_FT'] - df['Goals_H_FT']

df['Media_SG_H'] = df.groupby('Home')['SG_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_A'] = df.groupby('Away')['SG_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_H'] = df.groupby('Home')['Media_SG_H'].shift(1)
df['Media_SG_A'] = df.groupby('Away')['Media_SG_A'].shift(1)

df['DesvPad_SG_H'] = df.groupby('Home')['SG_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_A'] = df.groupby('Away')['SG_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_H'] = df.groupby('Home')['DesvPad_SG_H'].shift(1)
df['DesvPad_SG_A'] = df.groupby('Away')['DesvPad_SG_A'].shift(1)

df['CV_SG_H'] = df['DesvPad_SG_H'] / df['Media_SG_H']
df['CV_SG_A'] = df['DesvPad_SG_A'] / df['Media_SG_A']

In [ ]:
display(df)

### Variáveis de Desempenho Ponderadas Pela Odd

##### Média de Saldo de Gols Ponderado pela Probabilidade do Time

In [ ]:
df['SG_H_01'] = (df['Goals_H_FT'] - df['Goals_A_FT']) / df['p_H']
df['SG_A_01'] = (df['Goals_A_FT'] - df['Goals_H_FT']) / df['p_A']

df['Media_SG_H_01'] = df.groupby('Home')['SG_H_01'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_A_01'] = df.groupby('Away')['SG_A_01'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_H_01'] = df.groupby('Home')['Media_SG_H_01'].shift(1)
df['Media_SG_A_01'] = df.groupby('Away')['Media_SG_A_01'].shift(1)

df['DesvPad_SG_H_01'] = df.groupby('Home')['SG_H_01'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_A_01'] = df.groupby('Away')['SG_A_01'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_H_01'] = df.groupby('Home')['DesvPad_SG_H_01'].shift(1)
df['DesvPad_SG_A_01'] = df.groupby('Away')['DesvPad_SG_A_01'].shift(1)

df['CV_SG_H_01'] = df['DesvPad_SG_H_01'] / df['Media_SG_H_01']
df['CV_SG_A_01'] = df['DesvPad_SG_A_01'] / df['Media_SG_A_01']

##### Média de Saldo de Gols Ponderado pela Probabilidade do Adversário

In [ ]:
df['SG_H_02'] = (df['Goals_H_FT'] - df['Goals_A_FT']) / df['p_A']
df['SG_A_02'] = (df['Goals_A_FT'] - df['Goals_H_FT']) / df['p_H']

df['Media_SG_H_02'] = df.groupby('Home')['SG_H_02'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_A_02'] = df.groupby('Away')['SG_A_02'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_H_02'] = df.groupby('Home')['Media_SG_H_02'].shift(1)
df['Media_SG_A_02'] = df.groupby('Away')['Media_SG_A_02'].shift(1)

df['DesvPad_SG_H_02'] = df.groupby('Home')['SG_H_02'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_A_02'] = df.groupby('Away')['SG_A_02'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_H_02'] = df.groupby('Home')['DesvPad_SG_H_02'].shift(1)
df['DesvPad_SG_A_02'] = df.groupby('Away')['DesvPad_SG_A_02'].shift(1)

df['CV_SG_H_02'] = df['DesvPad_SG_H_02'] / df['Media_SG_H_02']
df['CV_SG_A_02'] = df['DesvPad_SG_A_02'] / df['Media_SG_A_02']

##### Média do Valor do Gol

In [ ]:
df['VG_H'] = df['Goals_H_FT'] * df['p_A']
df['VG_A'] = df['Goals_A_FT'] * df['p_H']

df['Media_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_VG_H'] = df.groupby('Home')['Media_VG_H'].shift(1)
df['Media_VG_A'] = df.groupby('Away')['Media_VG_A'].shift(1)

df['DesvPad_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_VG_H'] = df.groupby('Home')['DesvPad_VG_H'].shift(1)
df['DesvPad_VG_A'] = df.groupby('Away')['DesvPad_VG_A'].shift(1)

df['CV_VG_H'] = df['DesvPad_VG_H'] / df['Media_VG_H']
df['CV_VG_A'] = df['DesvPad_VG_A'] / df['Media_VG_A']

##### Custo do Gol

In [ ]:
# Custo do Gol 1.0

df['CG_H_01'] = df['Goals_H_FT'] / df['p_H']
df['CG_A_01'] = df['Goals_A_FT'] / df['p_A']

df['Media_CG_H_01'] = df.groupby('Home')['CG_H_01'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_A_01'] = df.groupby('Away')['CG_A_01'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_H_01'] = df.groupby('Home')['Media_CG_H_01'].shift(1)
df['Media_CG_A_01'] = df.groupby('Away')['Media_CG_A_01'].shift(1)

df['DesvPad_CG_H_01'] = df.groupby('Home')['CG_H_01'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_A_01'] = df.groupby('Away')['CG_A_01'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_H_01'] = df.groupby('Home')['DesvPad_CG_H_01'].shift(1)
df['DesvPad_CG_A_01'] = df.groupby('Away')['DesvPad_CG_A_01'].shift(1)

df['CV_CG_H_01'] = df['DesvPad_CG_H_01'] / df['Media_CG_H_01']
df['CV_CG_A_01'] = df['DesvPad_CG_A_01'] / df['Media_CG_A_01']

In [ ]:
# Custo do Gol 2.0

df['CG_H_02'] = (df['Goals_H_FT'] / 2) + (df['p_H'] / 2)
df['CG_A_02'] = (df['Goals_A_FT'] / 2) + (df['p_A'] / 2)

df['Media_CG_H_02'] = df.groupby('Home')['CG_H_02'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_A_02'] = df.groupby('Away')['CG_A_02'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_H_02'] = df.groupby('Home')['Media_CG_H_02'].shift(1)
df['Media_CG_A_02'] = df.groupby('Away')['Media_CG_A_02'].shift(1)

df['DesvPad_CG_H_02'] = df.groupby('Home')['CG_H_02'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_A_02'] = df.groupby('Away')['CG_A_02'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_H_02'] = df.groupby('Home')['DesvPad_CG_H_02'].shift(1)
df['DesvPad_CG_A_02'] = df.groupby('Away')['DesvPad_CG_A_02'].shift(1)

df['CV_CG_H_02'] = df['DesvPad_CG_H_02'] / df['Media_CG_H_02']
df['CV_CG_A_02'] = df['DesvPad_CG_A_02'] / df['Media_CG_A_02']

In [ ]:
display(df)

In [ ]:
df = drop_reset_index(df)
display(df)

In [ ]:
df = df.sort_values(by='Date')
df = drop_reset_index(df)
display(df)

In [ ]:
df.to_excel("Base_de_Dados_com_Variáveis.xlsx", index=False)